In [ ]:
import altair as alt
import geopandas as gpd

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

#### Barcelona map source

https://opendata-ajuntament.barcelona.cat/data/en/dataset/20170706-districtes-barris/resource/cd800462-f326-429f-a67a-c69b7fc4c50a

First line of `0301100100_UNITATS_ADM_POLIGONS.json`

In [ ]:
barcelona_identity_gdf = gpd.read_parquet("data/barcelona_geometry.parquet")

geopandas.geodataframe.GeoDataFrame

In [8]:
# This creates a new GeoDataFrame with the geometries transformed to Mercator
barcelona_mercator_gdf = barcelona_identity_gdf.to_crs(epsg=4326)

print(f"Reprojected CRS: {barcelona_mercator_gdf.crs}")

barcelona_mercator_gdf.head()

Reprojected CRS: EPSG:4326


,FID,ID_ANNEX,ANNEXDESCR,ID_TEMA,TEMA_DESCR,ID_CONJUNT,CONJ_DESCR,ID_SUBCONJ,SCONJ_DESC,ID_ELEMENT,...,CODI_UA,TIPUS_UA,NOM,WEB1,WEB2,WEB3,FHEX_COLOR,Shape_Leng,Shape_Area,geometry
0,0,01,Grup - I,0104,Unitats Administratives,010411,Terme Municipal,01041101,Terme Municipal,0104110101,...,080193,TERME,Barcelona,http://www.bcn.cat,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/document...,#000000,113967.553914,1.017022e+08,"MULTIPOLYGON (((2.18706 41.46772, 2.1858 41.46..."


In [9]:
seq_gdf = gpd.read_parquet("data/h3geo_hourly_percentage_docks_available.parquet")

print(seq_gdf.shape)

seq_gdf.head()

(2376, 7)


,timestamp,h3_index_resolution_8,geojson,day_of_year,time_seq_group,seq_type,percentage_docks_available
0,2024-11-25 00:00 (Mon),8839446001fffff,"POLYGON ((2.13052 41.40655, 2.12933 41.40179, ...",330,1,avg,0.870679
1,2024-11-25 00:00 (Mon),8839446003fffff,"POLYGON ((2.14105 41.40549, 2.13986 41.40072, ...",330,1,avg,0.707031
2,2024-11-25 00:00 (Mon),8839446005fffff,"POLYGON ((2.12348 41.39993, 2.12229 41.39517, ...",330,1,avg,0.847597
3,2024-11-25 00:00 (Mon),8839446007fffff,"POLYGON ((2.134 41.39887, 2.13282 41.39411, 2....",330,1,avg,0.828447
4,2024-11-25 00:00 (Mon),8839446009fffff,"POLYGON ((2.12703 41.41423, 2.12584 41.40946, ...",330,1,avg,0.882716


In [10]:
background = alt.Chart(barcelona_mercator_gdf).mark_geoshape(
    fill='lightgray',
    stroke='white',
).project(
    'identity',
    reflectY=True,
).properties(
    width=600,
    height=500, # Adjust height/width as needed for aspect ratio
)

background

alt.Chart(...)

In [11]:
# X-value slider
x_slider = alt.binding_range(min=int(seq_gdf["time_seq_group"].min()), max=int(seq_gdf["time_seq_group"].max()), step=1, name='time_seq_group')
x_select = alt.selection_point(name="x_select", fields=['time_seq_group'], bind=x_slider, value=1)

color_scale = alt.Scale(
    scheme="redyellowgreen",
    domainMin=0,
    domainMax=1,
)

base = alt.Chart(seq_gdf).mark_geoshape(stroke="black", strokeWidth=0.5).encode(
    color=alt.Color("percentage_docks_available", title="Percentage Docks Available").scale(color_scale),
).transform_filter(
    x_select
).add_params(
    x_select
).project("identity", reflectY=True)

background_year = alt.Chart(seq_gdf).mark_text(
    align="left",
    baseline="top",
    dx=-290,
    dy=-240,
    fontSize=32,
    color="lightslategrey",
    opacity=0.2
).encode(
    text='timestamp'
).transform_filter(
    x_select
)

chart = alt.layer(
    background,
    base,
    background_year,
)
chart

alt.LayerChart(...)

In [12]:
chart.save('h3geo_altair_interactive_chart.html')